In [ ]:
"""
Date   : Nov 12 2017
@author: Archana Neelipalayam Masilamani
Project Description:
Implemented Nearest Neighbor Recommender System
 
The dataset used is MovieLens.

The task is to recommend movie to a user based on the ratings of the user and other users
K-Nearest Neighborrecommandation method is followed
 
"""



In [62]:

import pandas as pd
import numpy as np 
from scipy.spatial.distance import correlation 

#Read the movie.csv and ratings.csv files

ratings_data = pd.read_csv("ml-latest-small/ratings.csv",
                           index_col=False)
movies_data  = pd.read_csv("ml-latest-small/movies.csv",
                           index_col=False, usecols=[0,1])

#Merging movie and rating data
data = ratings_data.merge(movies_data,left_on = 'movieId', right_on = 'movieId')

data.head()

,userId,movieId,rating,timestamp,title
0,1,31,2.5,1260759144,Dangerous Minds (1995)
1,7,31,3.0,851868750,Dangerous Minds (1995)
2,31,31,4.0,1273541953,Dangerous Minds (1995)
3,32,31,4.0,834828440,Dangerous Minds (1995)
4,36,31,3.0,847057202,Dangerous Minds (1995)


In [4]:
#Create a pivot table such that rows are UserId's, columns are all set of movies, and values
#are rating given by the user for that movie. If a rating is not available, then the
#value will be NaN

data_table = pd.pivot_table(data, values='rating',index ='userId',columns='movieId')

data_table.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# To find the K nearest neighbors, the similarity between
# two users has to be found. Correlation function from
# 'scipy' library can be used for this purpose

def similarity(user1,user2):   
# Normalize the ratings of user1 and user2 by mean ratings of user1
# for any movie
    user1 = np.array(user1) - np.nanmean(user1)
    user2 = np.array(user2) - np.nanmean(user2)
# The similarty between two users is found only using the rating data
# for movies that has been rated by both the users in common

    cominItemId  = [i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]

    if len(cominItemId) == 0:
        return 0;
    else:
        user1 = np.array([user1[i] for i in cominItemId])
        user2 = np.array([user2[i] for i in cominItemId])
    return correlation(user1,user2)

# correlation(np.array(data_table.loc[1]),np.array(data_table.loc[2]))
 
    
# Function to find the ratings of the nearest neighbors

def kNearestNeighbor(K,currentuser):
    # Create a dataframe which will be used to hold the similarity between the
    # current user and all the other users -> Rows will be all users and 
    #column will be the similarity value between the current user and all users
    similarityDataframe = pd.DataFrame(index = data_table.index, columns = 
                                       ['SimilarityVal'])
    
    #Call the function Similarity between the current user and all the other users
    for i in data_table.index:
        similarityDataframe.loc[i] = similarity(data_table.loc[currentuser],data_table.loc[i])
        
    # Sort the similarity Dataframe    
    similarityDataframe = pd.DataFrame.sort_values(similarityDataframe,
                                          ['SimilarityVal'],ascending=[0])                                                             
    
    nearestNeighbors = similarityDataframe[:K]
    
    return nearestNeighbors
    

# The following script is to test the function for sample data where K is 3 and user id is 1
nearestNeighborList = kNearestNeighbor(3,1)
nearestNeighborList


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/scipy/spatial/distance.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(um, vm) / (norm(um) * norm(vm))


,SimilarityVal
userId,
468,2
195,2
564,2


In [25]:
# This script is used in the below function 
neighborRatings = data_table.loc[nearestNeighborList.index]
neighborRatings.head()



movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
468,4.0,2.0,2.5,NaN,NaN,3.5,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
564,4.0,4.0,3.0,NaN,3.0,1.0,3.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Function to find the predicted ratings based on the K-Nearest
# neighbors(by applying the formula). The weight in formula is nothing
# but the similarity value between the user and the neighbor

#First take all the ratings of the nearest neighbors
def predictedRatings(K,currentUser):    
    
    nearestNeighborList = kNearestNeighbor(K,currentUser)
    
    neighborRatings = data_table.loc[nearestNeighborList.index]
    neighborRatings.head()
    # Create a final predicted ratings matrix, where the 
    # rows will be the movies and column will be the predicted ratings

    predictedItemRating = pd.DataFrame(index = data_table.columns,
                                  columns = ['Rating'])
    # Refer formula
    # For every movie 
    for  i in data_table.columns:
        
    #Find the mean of rating of the current user(mean of 'ra'
    #as per the formula)
        currentUserMeanRating = np.nanmean(data_table.loc[currentUser])
        predictedRating = currentUserMeanRating
        ratingsSum = 0
        weightSum = 0
        #For every Neighbor
        for j in neighborRatings.index:
            weightSum += nearestNeighborList.loc[j,'SimilarityVal']
            #If the rating of that movie is given by the neighbor
            if neighborRatings.loc[j,i]>0:  
             # As per the formula, inside the summation, for every user,
             # we have to sum the product of weight(similarity) between the
             # currrent user & the neighbor and the difference between the 
             # rating of the neighbor to the mean of ratings of the neighbor
             # This is added to the mean ratings of current user for all movies
             # In the below formula, we are not normalizing with sum of similarity
             # or weights between the users. But that can be added if required
             ratingsSum += (neighborRatings.loc[j,i]                                   
                  - np.nanmean(neighborRatings.loc[j]))*nearestNeighborList.loc[j,'SimilarityVal']
        
            #Normalizing the weights
            predictedRating  = predictedRating + (ratingsSum/weightSum)           
        # Adding the predicted rating calculated to the predictedItemRating    
        predictedItemRating.loc[i] =  predictedRating
    
    return predictedItemRating

# The following script is to test the function for sample data where K is 3 and user id is 1
predictedRatingsR = predictedRatings(3,1)
pd.DataFrame.sort_values(predictedRatingsR,['Rating'],ascending=[0]).head()


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/scipy/spatial/distance.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(um, vm) / (norm(um) * norm(vm))


,Rating
movieId,
1252,4.06106
910,4.06106
1278,3.89439
1214,3.89439
1304,3.89439


In [44]:
#This script gives the movies already watched by user 3. Its 
#used in the below function
data_table.loc[3].loc[data_table.loc[3]>0].head()


movieId
60     3.0
110    4.0
247    3.5
267    3.0
296    4.5
Name: 3, dtype: float64

In [87]:
# Function to use the ablove predicted ratings to recommend the
# top N movies for the user

def movieRecommendation(N,currentUser):
    # Here we take K = 10, for the number of nearest neighbors
    predictedRatingList = predictedRatings(10,currentUser)
    # Extract the list of movies already watched by the current user
    # since we do not have to include it for the recommendation
    moviesWatchedByCurrentUser = list(data_table.loc[currentUser]
                                     .loc[data_table.loc[currentUser]>0].index)
    # Remove the details of already watched movies from 
    # the predictedRatingList
    predictedRatingList = predictedRatingList.drop(moviesWatchedByCurrentUser)
    
    # Now sort the predictedRatingList to find the top N predicted ratings
    
    topMovies = pd.DataFrame.sort_values(predictedRatingList,['Rating'],ascending=[0])[:N]
    
    # Extract the movie titles for the top N movies
    topMoviesTitle = movies_data.loc[movies_data.movieId.isin(topMovies.index)]
    
    return topMoviesTitle
   
currentUser = 80
N = 10
result = movieRecommendation(N,currentUser)['title']

#Remove the index from the dataframe by converting it to a string
result = result.to_string(index=False)

print("The top ", N, " movies recommandation for the user ",currentUser, ":\n")


#Print to top N movies
for movie in result.split('\n'):
    print(movie.strip())


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/scipy/spatial/distance.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(um, vm) / (norm(um) * norm(vm))


The top  10  movies recommandation for the user  80 :

Braveheart (1995)
Apollo 13 (1995)
Star Wars: Episode IV - A New Hope (1977)
Shawshank Redemption, The (1994)
Primal Fear (1996)
Ransom (1996)
Contact (1997)
Game, The (1997)
Sixth Sense, The (1999)
The Butterfly Effect (2004)


In [ ]:
# End